In [1]:
import tkinter as tk

In [6]:
!pip3.12 install librosa scikit-learn torch torchaudio --break-system-packages

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 28.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 32.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.1 MB/s eta 0:00:00a 0:00:01
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [25]:
import torchaudio
import torch
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

wf1_resampled, sr1 = torchaudio.load("../Data/240519 0500 Subject 1.wav")
wf1_resampled = np.array(wf1_resampled)

In [26]:
frame_size = int(0.5 * sr1)  # 0.5 seconds
hop_size = int(0.25 * sr1)  # 0.25 seconds
frames = []
for i in range(0, len(wf1_resampled), hop_size):
    frame = wf1_resampled[i:i + frame_size]
    frames.append(frame)

In [27]:
def extract_mfcc(frame):
    mfccs = librosa.feature.mfcc(y=frame, sr=sr1, n_mfcc=13)
    return mfccs

features = np.array([extract_mfcc(frame) for frame in frames])
features = features.reshape(features.shape[0], -1)  # Reshape to 2D array

: 

: 

In [6]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)[0].reshape(-1, 1)[:300000] # out of 14372943

In [23]:
features

array([[-1131.3708, -1131.3708, -1131.3708, ...,     0.    ,     0.    ,
            0.    ]], dtype=float32)

In [7]:
len(features_scaled)

300000

In [21]:
features_scaled.reshape(1, -1)[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [22]:
from collections import Counter
Counter(list(features_scaled.reshape(1, -1)[0]))

Counter({0.0: 300000})

In [24]:
Counter(list(features.reshape(1, -1)[0]))

Counter({0.0: 13267332, -1131.3708: 1105611})

In [ ]:
features_scaled = features_scaled.drop_duplicates()

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_scaled, np.zeros(len(features_scaled)), test_size=0.1, random_state=42)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, max_iter = 1000, init='k-means++', random_state=42)
kmeans.fit(features_scaled)

y_pred = kmeans.labels_

/opt/homebrew/lib/python3.12/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [12]:
from collections import Counter
distribution = Counter(y_pred)
print(distribution)

Counter({0: 300000})


In [16]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'

In [17]:
import matplotlib.pyplot as plt

# Downsample the data (optional)
# wf1_resampled = wf1_resampled[::10]  # downsample by 10
# y_pred = y_pred[::10]

plt.figure(figsize=(12, 6))

# Plot the original audio signal
plt.subplot(2, 1, 1)
plt.plot(wf1_resampled, 'bo')  # faster plotting style
plt.title("Original Audio Signal")
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")

# Plot the classification results
plt.subplot(2, 1, 2)
plt.plot(y_pred, 'bo')  # faster plotting style
plt.title("KNN Classification Results")
plt.xlabel("Frame Index")
plt.ylabel("Class Label")

plt.tight_layout()
plt.show()

: 

: 